# Example1: Load flow as a function with inputs

In [ ]:
from promptflow import load_flow


flow_path = "../../flows/standard/web-classification"
sample_url = "https://www.youtube.com/watch?v=o5ZQyXaAv1g"

f = load_flow(source=flow_path)
result = f(url=sample_url)

print(result)

# Example2: Load flow as a function with connection overwrite

You will need to have a connection named "new_ai_connection" to run flow with new connection.

In [ ]:
# create needed connection
import promptflow
from promptflow.entities import AzureOpenAIConnection, OpenAIConnection

pf = promptflow.PFClient()

try:
    conn_name = "new_ai_connection"
    conn = pf.connections.get(name=conn_name)
    print("using existing connection")
except:
    # Follow https://learn.microsoft.com/en-us/azure/ai-services/openai/how-to/create-resource?pivots=web-portal to create an Azure Open AI resource.
    connection = AzureOpenAIConnection(
        name=conn_name,
        api_key="<user-input>",
        api_base="<test_base>",
        api_type="azure",
        api_version="<test_version>",
    )

    # use this if you have an existing OpenAI account
    # connection = OpenAIConnection(
    #     name=conn_name,
    #     api_key="<user-input>",
    # )
    conn = pf.connections.create_or_update(connection)
    print("successfully created connection")

print(conn)

In [ ]:
f = load_flow(
    source=flow_path,
)
# need to create the connection
f.context.connections = {"classify_with_llm": {"connection": "new_ai_connection"}}

result = f(url=sample_url)

print(result)

## Example 3: Local flow as a function with flow inputs overwrite

In [ ]:
from promptflow.entities import FlowContext

f = load_flow(source=flow_path)
f.context = FlowContext(
    # node "fetch_text_content_from_url" will take inputs from the following command instead of from flow input
    overrides={"nodes.fetch_text_content_from_url.inputs.url": sample_url},
)
# the url="unknown" will not take effect
result = f(url="unknown")
print(result)

## Example 4: Load flow as a function with streaming output

In [ ]:
f = load_flow(source="../../flows/chat/basic-chat")
f.context.streaming = True
result = f(
    chat_history=[
        {
            "inputs": {"chat_input": "Hi"},
            "outputs": {"chat_output": "Hello! How can I assist you today?"},
        }
    ],
    question="How are you?",
)


answer = ""
# the result will be a generator, iterate it to get the result
for r in result["answer"]:
    answer += r

print(answer)